### Import libraries

In [68]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
import keras_tuner as kt
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Set seed for reproducibility

In [69]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Exploration Data Analysis (EDA)

Load the dataset

In [70]:
model_dir = 'models'
dataset = pd.read_csv('../input/sequencean2dl/Training.csv', dtype='float32')
print(dataset.shape)
dataset.head()

In [71]:
dataset.info()

Sequential Train-Test split and normalization

In [72]:
val_size = 4000
X_train_raw = dataset.iloc[:-val_size]
X_val_raw = dataset.iloc[-val_size:]
print(X_train_raw.shape, X_val_raw.shape)

# Normalize both features and labels
X_min = tf.math.reduce_min(dataset, axis=0).numpy()
X_max = tf.math.reduce_max(dataset, axis=0).numpy()
X_std = tf.math.reduce_std(dataset, axis=0).numpy()

scale= X_max-X_min

X_train_raw = (X_train_raw-X_min)/scale
X_val_raw = (X_val_raw-X_min)/scale

def mloss(y_true, y_pred):
    wei = scale / tf.math.reduce_mean(scale)
    delta = wei * (y_true-y_pred)
    return tf.math.reduce_mean(tf.math.square(delta))

def eval_loss(y_true, y_pred):
    delta = scale * (y_true - y_pred)
    return tf.math.sqrt(tf.math.reduce_mean(tf.math.square(delta)))

figs, axs = plt.subplots(len(dataset.columns), 1, sharex=True, figsize=(17,17))
for i, col in enumerate(dataset.columns):
    axs[i].plot(X_train_raw[col])
    axs[i].plot(X_val_raw[col])
    axs[i].set_title(col)


In [73]:
window = 1024
stride = 16

In [74]:
def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Discard don't pad
        temp_df = temp_df[padding_len:]
        temp_label = temp_label[padding_len:]
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels


In [75]:
telescope = 144
target_labels = dataset.columns
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_val, y_val = build_sequences(X_val_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [76]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,0:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [77]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

## Model

In [78]:
input_shape = X_train.shape[1:] # (None, X_train.shape[2])
output_shape = y_train.shape[1:]
output_dim = output_shape[-1]

print(input_shape)
print(output_shape)

In [82]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates
    
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def create_look_ahead_mask(size, steps):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, tf.math.minimum(steps, size))
    return mask

def ffn(dim, dropout, width):
    return tfk.Sequential([
        tfkl.Conv1D(4 * dim, width, padding="causal"),
        tfkl.ELU(),
        tfkl.SpatialDropout1D(dropout),
        tfkl.Conv1D(dim, width, padding="causal"),
        tfkl.ELU(),
        tfkl.SpatialDropout1D(dropout),
    ])

class EncoderLayer(tfkl.Layer):
    def __init__(self, max_len, dims, heads, dropout=0.0, ffn_w=1):
        super(EncoderLayer, self).__init__()
        self.attention = tfkl.MultiHeadAttention(heads, dims, dropout=dropout)
        self.norm1 = tfkl.BatchNormalization(epsilon=1e-6)
        self.norm2 = tfkl.BatchNormalization(epsilon=1e-6)
        self.ffn = ffn(dims, dropout, ffn_w)
        self.add0 = tfkl.Add()
        self.add1 = tfkl.Add()
        self.add2 = tfkl.Add()
        self.mult = tfkl.Multiply()
        self.layers = [self.attention, self.norm1, self.norm2, self.ffn, self.add1, self.add2]
        self.scaling_factor = tf.keras.backend.constant(np.sqrt(dims), shape = (1,1,1))
        self.pos_enc = positional_encoding(max_len, dims)

        
    def call(self, x, training):
        x = self.add0([x, self.pos_enc[:, :tf.shape(x)[1], :]])
        x = self.mult([x, self.scaling_factor], training=training)
        
        res = x
        x = self.attention(x, x, training=training)
        x = self.add1([x, res])
        x = self.norm1(x, training=training)
        
        res = x
        x = self.ffn(x, training=training)
        x = self.add2([x, res])
        x = self.norm2(x, training=training)

        return x


class DecoderLayer(tfkl.Layer):
    def __init__(self, max_len, dims, heads, dropout=0.0, ffn_w=1):
        super(DecoderLayer, self).__init__()
        self.attention1 = tfkl.MultiHeadAttention(heads, dims, dropout=dropout)
        self.attention2 = tfkl.MultiHeadAttention(heads, dims, dropout=dropout)
        self.norm1 = tfkl.BatchNormalization(epsilon=1e-6)
        self.norm2 = tfkl.BatchNormalization(epsilon=1e-6)
        self.norm3 = tfkl.BatchNormalization(epsilon=1e-6)
        self.ffn = ffn(dims, dropout, ffn_w)
        self.layers = [self.attention1, self.attention2, self.norm1, self.norm2,self.norm3, self.ffn]
        self.scaling_factor = tf.keras.backend.constant(np.sqrt(dims), shape = (1,1,1))
        self.pos_enc = positional_encoding(max_len, dims)

        
    def call(self, inputs, training):
        x = inputs[0]
        y = inputs[1]
        
        x = tfkl.Add()([x, self.pos_enc[:, :tf.shape(x)[1], :]])
        x = tfkl.Multiply()([x, self.scaling_factor])
        
        y = tfkl.Add()([y, self.pos_enc[:, :tf.shape(y)[1], :]])
        y = tfkl.Multiply()([y, self.scaling_factor])
        
        res = x
        x = self.attention1(x, x, training=training)
        x = tfkl.Add()([x, res])
        x = self.norm1(x, training=training)
        
        x = tfkl.Multiply()([x, self.scaling_factor])
        
        res = x
        x = self.attention2(x, y, training=training)
        x = tfkl.Add()([x, res])
        x = self.norm2(x, training=training)
        
        res = x
        x = self.ffn(x, training=training)
        x = tfkl.Add()([x, res])
        x = self.norm3(x, training=training)

        return x

def build_model(hp=None):
    dims = 96 # hp.Int("dimensions", 48, 64, default=56)
    dropout = 0.25 # hp.Float("dropout", 0.1, 0.4, default=0.37)
    encoders = 1 # hp.Int("encoders", 1, 2, default=1)
    heads = 8 # hp.Int("heads", 6, 10, default=8)
    lr = 0.001 # hp.Float("lr_hp", 0.0005, 0.002, default=0.001)
    conv_width = 3 # hp.Int("conv_width", 2, 5, default=3)
    strides = 2 # hp.Choice("strides", [2, 3, 4])
    ffn_w = 1 # hp.Int("ffn_width", 2, 4)
    noise = 0.01 # hp.Float("noise", 0.001, 0.05)
    
    input_past = tfkl.Input(shape=(None, 7))
    past_noised = tfkl.GaussianNoise(noise)(input_past)
    max_pos_enc = input_shape[0] + output_shape[0]

    x = past_noised
    x = tfkl.Conv1D(dims, conv_width, padding="causal", strides=strides)(x)

    for _ in range(encoders):
        x = EncoderLayer(max_pos_enc, dims, heads, dropout, ffn_w)(x)

    x = tfkl.Conv1D(output_shape[1], 1, padding="causal")(x)
    
    transformer_outputs = x[:, -telescope:, :]
    transformer = tfk.Model(inputs=input_past, outputs=transformer_outputs)
    
    transformer.compile(loss=mloss, optimizer=tfk.optimizers.Adam(lr, epsilon=1e-6), metrics=['mse', eval_loss, 'mae'])

    return transformer
    

## Hyperparameter search

In [ ]:
"""
import keras_tuner as kt
tuner = kt.Hyperband(lambda hp: build_model(hp),
                     loss=mloss,
                     objective='val_loss',
                     max_epochs=24,
                     factor=3,
                     hyperband_iterations=1,
                     overwrite=True,
                     directory='kt_dir',
                     project_name='kt0')

tuner.search(
    x = X_train,
    y = y_train,
    shuffle=True,
    batch_size = 64,
    epochs = 100,
    validation_data = (X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True),
        tfk.callbacks.TensorBoard("kt_dir/tb_logs/")
    ]
)

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

tuner.results_summary(10)
for i, h in enumerate(tuner.get_best_hyperparameters(5)):
    print(i)
    for k in h.values:
        print(f"{k}: {h.values[k]}")
    print()
"""

In [91]:
"""
epochs = 32
batch_size = 64

model.compile(loss=mloss, optimizer=tfk.optimizers.Adam(0.0001, epsilon=1e-6), metrics=['mse', eval_loss, 'mae'])

# Train the model
model.fit(
    x = X_train,
    y = y_train,
    shuffle=True,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', patience=4, factor=0.2, min_lr=1e-5),
        # tfk.callbacks.TensorBoard('tb_dir/', profile_batch=0, histogram_freq=1),
    ]
)
"""

## Training

In [ ]:
model = build_model()

In [ ]:
epochs = 100
batch_size = 64

# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    shuffle=True,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=18, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', patience=5, factor=0.2, min_lr=1e-5),
        tfk.callbacks.TensorBoard('tb_dir/', profile_batch=0, histogram_freq=1),
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [95]:
model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(0.001, epsilon=1e-6), metrics=['mae'])
model_name = 'SubOptimusTime144'
now = datetime.now().strftime('_%b%d_%H-%M-%S')
model_path = os.path.join(model_dir, model_name + now)
weights_path = os.path.join(model_dir, "weights", model_name + now, model_name)

model.save_weights(weights_path)
model.save(model_path)

In [96]:
!zip -rq "{model_name}.zip" "models/"

Test evaluation

In [ ]:
def inspect_multivariate_prediction(X, y, pred, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(pred))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].plot(np.arange(len(X[0,:,i]), len(X[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i]), len(X[0,:,i])+telescope), pred[idx,:,i], color='green')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [ ]:
predictions = model.predict(X_val)
inspect_multivariate_prediction(X_val, y_val, predictions, target_labels, telescope)

### Multivariate Forecasting (Autoregression)

In [85]:
reg_telescope = 864
X_val_reg, y_val_reg = build_sequences(X_val_raw, target_labels, window, stride, reg_telescope)
X_val_reg.shape, y_val_reg.shape

In [92]:
# Autoregressive Forecasting
reg_predictions = np.array([])
X_temp = X_val_reg
for reg in range(0,reg_telescope + telescope,telescope):
    pred_temp = model.predict(X_temp)
    if(len(reg_predictions)==0):
        reg_predictions = pred_temp
    else:
        reg_predictions = np.concatenate((reg_predictions,pred_temp),axis=1)
    X_temp = np.concatenate((X_temp[:,telescope:,:],pred_temp), axis=1)
    
reg_predictions = reg_predictions[:,:reg_telescope, :]
reg_predictions.shape

In [93]:
mean_squared_error = tfk.metrics.mse(y_val_reg.flatten(),reg_predictions.flatten())
mean_absolute_error = tfk.metrics.mae(y_val_reg.flatten(),reg_predictions.flatten())
eval_error = eval_loss(y_val_reg, reg_predictions)

mean_squared_error.numpy(), mean_absolute_error.numpy(), eval_error.numpy()

In [94]:
inspect_multivariate_prediction(X_val_reg, y_val_reg, reg_predictions, target_labels, reg_telescope)

## Inspect activations

In [ ]:

def all_layers(l):
    new_l = [l]
    layers = []
    while len(new_l) > 0:
        l = new_l.pop(0)
        layers.append(l)
        try:
            if l.layers is not None:
                for sl in l.layers:
                    new_l.append(sl)
        except AttributeError:
            continue
    return layers

layers = all_layers(model)

layers = [layer.output for layer in layers if isinstance(layer, (tfkl.Conv1D))]

activation_model = tf.keras.Model(inputs=model.input, outputs=layers)

# Finally we get the output feature maps (for each layer) given the imput test image
test = X_val[0]

fmaps = activation_model.predict(tf.expand_dims(test, 0))
import matplotlib.pyplot as plt
%matplotlib inline
def display_activation(fmaps, depth=0, first_n=-1): 
    # fmaps: list of all the feature maps for each layer
    # depth: the layer we want to visualize (an int in [0, network depth))
    # first_n: default '-1' means 'all activations'. Number of activations to be visualized. Note that for deep layers it could be a large number.

    fmaps = fmaps[depth] # get feature maps at the desired depth
    if first_n > 0:
        fmaps = fmaps[:, :first_n]

    # Distribute on a grid for plotting
    col_size = 4
    row_size = fmaps.shape[-1] // col_size + 1

    fig, axs = plt.subplots(row_size, col_size, sharex="all", sharey="all", figsize=(32,16))

    for fmap_channel in range(fmaps.shape[-1]):
        axs[fmap_channel//col_size, fmap_channel%col_size].plot(fmaps[-telescope:, fmap_channel])
    plt.show()


for i in range(len(layers)):
    print(layers[i].name)
    display_activation(fmaps=fmaps, depth=i, first_n=32)